In [198]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


In [195]:
training_data = pd.read_csv("cleaned_datasets/training_data.csv")

In [196]:
features = ["AGE","GP","MpG","PpG","RpG","ApG","SpG","BpG","TOpG"]
X = training_data[features]
y = training_data["FPpG_next"]


In [197]:
model = LinearRegression()
model.fit(X,y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [199]:
y_pred = model.predict(X)
mae = mean_absolute_error(y, y_pred)
print("Baseline MAE:", mae)


Baseline MAE: 4.228265850888525


In [201]:
seasons = sorted(training_data["SEASON"].unique())

In [205]:
results = []
for t in seasons[:-1]:
    train = training_data[training_data.SEASON <= t]
    test  = training_data[training_data.SEASON == t+1]
    Xtr, ytr = train[features], train["FPpG_next"]
    Xte, yte = test[features],  test["FPpG_next"]
    m = LinearRegression().fit(Xtr, ytr)
    yhat = m.predict(Xte)
    results.append({"train_upto": t, "test_year": t+1, "MAE": mean_absolute_error(yte, yhat)})
pd.DataFrame(results)

,train_upto,test_year,MAE
0,2020,2021,4.355374
1,2021,2022,4.436763
2,2022,2023,4.028403


In [207]:

naive = []
seasons = sorted(training_data["SEASON"].unique())
for t in seasons[:-1]:
    test = training_data[training_data.SEASON == t+1].copy()
    prev = (
        training_data[training_data.SEASON == t][["NAME","FPpG"]]
        .rename(columns={"FPpG":"prev_FPpG"})
    )
    test = test.merge(prev, on="NAME", how="left")
    m = test["prev_FPpG"].notna()  # keep only players with prior season
    mae = mean_absolute_error(test.loc[m,"FPpG_next"], test.loc[m,"prev_FPpG"])
    coverage = m.mean()  # fraction of test players covered by the naive baseline
    naive.append({"train_upto": t, "test_year": t+1, "MAE": mae, "coverage": coverage})

pd.DataFrame(naive)


,train_upto,test_year,MAE,coverage
0,2020,2021,6.203557,0.793333
1,2021,2022,5.713042,0.817391
2,2022,2023,5.812419,0.825112


In [209]:
test = all_data[all_data.SEASON == 2024].copy()
Xte = test[features]
yte = test["FPpG"]

m = LinearRegression().fit(Xtr, ytr)
yhat = m.predict(Xte)

mae_2024 = mean_absolute_error(yte, yhat)
print("2023-2024 MAE:", mae_2024)

2023-2024 MAE: 1.8970677116624692
